In [1]:
import os
import mne
import PyQt6
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import StratifiedKFold

from scipy import stats
from scipy.stats import ttest_rel
from scipy.stats import false_discovery_control

%matplotlib qt

In [2]:
folders = os.listdir('bipolar/')

fldr2label = {
    'control (160)': 0,
    'Биполярка депресс тип (35)': 4,
    'Биполярка маниакальный тип (36)': 5,
    'мания_психи(30)': 6,
    'циклотимия (13)': 7,
    'depressed_32-0(22)': 1,
    'depressed_32-1(44)': 2,
    'depressed_32-2(22)': 3,
}
og_files = []
zg_files = []
og_labels = []
zg_labels = []
for fldr in folders:
    pth =  'bipolar/' + fldr
    og_pths = [x for x in os.listdir(pth) if x.lower().endswith('.edf') and ('og.' in x.lower() or 'ог.' in x.lower() or '_ог' in x.lower() or 'eo' in x.lower())]
    zg_pths = [x for x in os.listdir(pth) if x.lower().endswith('.edf') and ('zg.' in x.lower() or 'зг.' in x.lower() or '_зг' in x.lower() or 'ec' in x.lower() or 'fon.' in x.lower() or 'eс' in x.lower())]
    for f in og_pths:
        og_files.append(pth + '/' + f)
        og_labels.append(fldr2label[fldr])
    for f in zg_pths:
        zg_files.append(pth + '/' + f)
        zg_labels.append(fldr2label[fldr])

print(f'Number of files for open eyes: {len(og_files)}')
print(f'Number of files for closed eyes: {len(zg_files)}')

Number of files for open eyes: 363
Number of files for closed eyes: 363


## Fractal dimensionality

In [3]:
from mne_features.univariate import compute_higuchi_fd

In [54]:
channels2use = ['Fp1', 'Fp2', 'F3', 'Fz', 'F4', 'F7', 'F8', 'T3', 'T4', 'C3', 'Cz', 'C4', 'T5', 'T6', 'P3', 'Pz', 'P4', 'O1', 'O2']
name_pairs = [('Fp1', 'Fp2'), ('F3', 'F4'), ('F7', 'F8'), ('C3', 'C4'), ('T3', 'T4'), ('P3', 'P4'), ('T5', 'T6'), ('O1', 'O2')]
idx_pairs = [(0, 1), (2, 4), (5, 6), (9, 11), (7, 8), (14, 16), (12, 13), (17, 18)]

def get_data_from_sample(sample):
    data = None
    sample = sample.filter(l_freq=1, h_freq=30, method='iir', verbose=False)
    channels = sample.ch_names
    to_drop = channels[19:]
    sample.drop_channels(to_drop)

    new_idx = []
    skip = False
    for ch in channels2use:
        found = False
        for k in range(19):
            if ch in channels[k]:
                new_idx.append(k)
                found = True
                break
        if not found:
            skip = True
            break
    if not skip:
        s_freq = int(sample.info['sfreq'])
        data = sample.get_data()[new_idx]
    return data

def compute_hfd(data, s_freq):
    # psd, freqs = mne.time_frequency.psd_array_welch(data, sfreq=s_freq, fmin=4, fmax=7, n_fft=1024, n_per_seg=s_freq, verbose=False)
    # psd_avg = np.mean(psd, axis=1)
    hfd = compute_higuchi_fd(data)
    return hfd

hfds_og, hfds_zg = [], []
labels_og, labels_zg = [], []
for i in tqdm(range(len(og_files))):
    path_og = og_files[i]
    path_zg = zg_files[i]
    try:
        sample_og = mne.io.read_raw_edf(path_og, verbose=False, preload=True)
    except Exception as e:
        print(f'skipped {path_og}')
        continue

    sample_zg = mne.io.read_raw_edf(path_zg, verbose=False, preload=True)
    s_freq_og = int(sample_og.info['sfreq'])
    s_freq_zg = int(sample_zg.info['sfreq'])
    data_og = get_data_from_sample(sample_og)
    data_zg = get_data_from_sample(sample_zg)
    if data_og is None or data_zg is None:
        print(f'skipped {path_og} and {path_zg}')
    else:
        max_len = max(data_og.shape[1], data_zg.shape[1])
        data_og = data_og[:, :max_len]
        data_zg = data_zg[:, :max_len]

        hfd_og = compute_hfd(data_og, s_freq_og)
        hfd_zg = compute_hfd(data_zg, s_freq_zg)
        hfds_og.append(hfd_og)
        hfds_zg.append(hfd_zg)
        labels_og.append(og_labels[i])
        labels_zg.append(zg_labels[i])

  0%|          | 0/363 [00:00<?, ?it/s]

skipped bipolar/control (160)/BORUTTO_JANNA_VLADIMIROVNA_48_EO_free.edf and bipolar/control (160)/BORUTTO_JANNA_VLADIMIROVNA_48_EC_free.edf
skipped bipolar/control (160)/MANUILOVA_ELENA_55_EO_free.edf and bipolar/control (160)/MANUILOVA_ELENA_55_EC_free.edf
skipped bipolar/control (160)/Martinenko_45_EO.edf and bipolar/control (160)/Martinenko_45_EC_free.edf
skipped bipolar/control (160)/Skopincev_20_EO_free.edf


In [55]:
hfds_og = np.array(hfds_og)
hfds_zg = np.array(hfds_zg)
labels_og = np.array(labels_og)
labels_zg = np.array(labels_zg)

In [24]:
false_discovery_control(stats.shapiro(np.array(hfds_og), axis=0)[1])

array([1.97042369e-01, 1.15007669e-02, 2.78867799e-01, 4.47340213e-02,
       4.43026645e-02, 4.88909155e-01, 4.47340213e-02, 4.47340213e-02,
       1.00439038e-01, 8.42394391e-03, 4.13654059e-03, 7.83878194e-05,
       1.33173331e-15, 1.32786122e-09, 2.30017568e-03, 1.52739617e-03,
       2.26997344e-05, 4.47340213e-02, 6.85711054e-02])

In [25]:
false_discovery_control(stats.shapiro(np.array(hfds_zg), axis=0)[1])

array([6.23038485e-09, 2.84023729e-10, 2.18353361e-09, 1.62618298e-07,
       8.66755262e-09, 5.46648485e-08, 3.81706424e-09, 7.13256482e-07,
       5.90811479e-06, 8.12276451e-10, 1.40572404e-09, 5.17400899e-11,
       2.44687624e-18, 1.67933982e-17, 1.70895276e-12, 4.00268535e-11,
       3.79124049e-16, 1.92673127e-10, 2.68680294e-13])

Seems like they are not normally distributed, thus, I will use Mann-Whitney U-test.

### Open vs closed eyes

#### All electrodes

In [30]:
mw = stats.mannwhitneyu(np.array(hfds_og), np.array(hfds_zg))[1]
print(false_discovery_control(mw))

[1.08659730e-38 2.74893629e-42 4.16625859e-42 2.74893629e-42
 1.16818906e-43 2.96727873e-41 1.47514386e-42 9.23215257e-27
 5.05214039e-33 3.00140400e-31 1.40349717e-38 3.95938592e-37
 8.58084340e-37 4.27725142e-44 2.96727873e-41 4.27725142e-44
 1.46527028e-45 6.97533487e-40 4.20507025e-41]


In [45]:
mw = stats.mannwhitneyu(np.array(hfds_og), np.array(hfds_zg), alternative='greater')[1]
print(false_discovery_control(mw))

print(f'MAPE by channels: {np.mean(np.abs(hfds_og - hfds_zg) / hfds_og, axis=0) * 100}')

[5.43298652e-39 1.37446815e-42 2.08312929e-42 1.37446815e-42
 5.84094532e-44 1.48363937e-41 7.37571932e-43 4.61607628e-27
 2.52607020e-33 1.50070200e-31 7.01748587e-39 1.97969296e-37
 4.29042170e-37 2.13862571e-44 1.48363937e-41 2.13862571e-44
 7.32635138e-46 3.48766743e-40 2.10253512e-41]
MAPE by channels: [6.64518698 7.09615112 6.67046605 6.14305566 6.68949514 6.6731826
 6.95754987 6.50076435 6.24285337 5.71776771 5.90256885 5.85261238
 7.05363066 7.38651454 6.49860091 6.34692445 6.55006378 7.35559852
 7.12583097]


#### Combined by zones

In [46]:
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

mws = []
for zone in zone2ids.keys():
    print(f'ZONE: {zone}')
    zone_og = np.mean(hfds_og[:, zone2ids[zone]], axis=1)
    zone_zg = np.mean(hfds_zg[:, zone2ids[zone]], axis=1)
    mw = stats.mannwhitneyu(zone_og, zone_zg)[1]
    mws.append(mw)
    print(f'MAPE: {np.mean(np.abs(zone_og - zone_zg) / zone_og, axis=0) * 100 :.3f}')
    print()

print(false_discovery_control(mws))

ZONE: frontal
MAPE: 6.570

ZONE: temporal
MAPE: 6.538

ZONE: central
MAPE: 5.719

ZONE: parietal
MAPE: 6.404

ZONE: occipital
MAPE: 7.162

[1.67348318e-45 1.50879052e-40 1.16058753e-37 1.60206538e-45
 1.54981670e-41]


In [47]:
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

mws = []
for zone in zone2ids.keys():
    zone_og = np.mean(hfds_og[:, zone2ids[zone]], axis=1)
    zone_zg = np.mean(hfds_zg[:, zone2ids[zone]], axis=1)
    mw = stats.mannwhitneyu(zone_og, zone_zg, alternative='greater')[1]
    mws.append(mw)

print(false_discovery_control(mws))

[8.36741591e-46 7.54395258e-41 5.80293767e-38 8.01032690e-46
 7.74908352e-42]


#### Left vs right vs center

In [48]:
side2ids = {'right': [1, 4, 6, 8, 11, 13, 16, 18], 'left': [0, 2, 5, 7, 9, 12, 14, 17], 'center': [3, 10, 15]}
mws = []
for side in side2ids.keys():
    side_og = np.mean(hfds_og[:, side2ids[side]], axis=1)
    side_zg = np.mean(hfds_zg[:, side2ids[side]], axis=1)
    mw = stats.mannwhitneyu(side_og, side_zg, alternative='greater')[1]
    mws.append(mw)
    print(f'MAPE: {np.mean(np.abs(side_og - side_zg) / side_og, axis=0) * 100 :.3f}')

print(false_discovery_control(mws))

MAPE: 6.581
MAPE: 6.458
MAPE: 6.025
[2.71430623e-47 2.48949680e-43 1.21363121e-45]


#### Avg

In [50]:
avg_og = np.mean(hfds_og, axis=1)
avg_zg = np.mean(hfds_zg, axis=1)
mw = stats.mannwhitneyu(avg_og, avg_zg)[1]
print(mw)
print(f'MAPE: {np.mean(np.abs(avg_og - avg_zg) / avg_og, axis=0) * 100 :.3f}')

9.542281993486255e-47
MAPE: 6.425


### By diagnoses

Open eyes

In [63]:
# simple mean of all channels
groups = []

for diag, id in fldr2label.items():
    grp = hfds_og[labels_og == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=22.48018102574708, pvalue=0.002098542072566335)


In [71]:
diags = list(fldr2label.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control (160) vs Биполярка депресс тип (35): pvalue 0.396
control (160) vs Биполярка маниакальный тип (36): pvalue 0.107
control (160) vs мания_психи(30): pvalue 0.107
control (160) vs циклотимия (13): pvalue 0.289
control (160) vs depressed_32-0(22): pvalue 0.883
control (160) vs depressed_32-1(44): pvalue 0.017
control (160) vs depressed_32-2(22): pvalue 0.911
Биполярка депресс тип (35) vs Биполярка маниакальный тип (36): pvalue 0.533
Биполярка депресс тип (35) vs мания_психи(30): pvalue 0.465
Биполярка депресс тип (35) vs циклотимия (13): pvalue 0.155
Биполярка депресс тип (35) vs depressed_32-0(22): pvalue 0.396
Биполярка депресс тип (35) vs depressed_32-1(44): pvalue 0.387
Биполярка депресс тип (35) vs depressed_32-2(22): pvalue 0.543
Биполярка маниакальный тип (36) vs мания_психи(30): pvalue 0.883
Биполярка маниакальный тип (36) vs циклотимия (13): pvalue 0.017
Биполярка маниакальный тип (36) vs depressed_32-0(22): pvalue 0.301
Биполярка маниакальный тип (36) vs depressed_32-1(44

In [79]:
# larger classes
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_og = np.array([label_map[i] for i in labels_og])

groups = []
for diag, id in new_classes.items():
    grp = hfds_og[new_labels_og == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=14.402605085959646, pvalue=0.006115005841194129)


In [74]:
diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control vs depressed: pvalue 0.065
control vs bipolar: pvalue 0.054
control vs mania: pvalue 0.054
control vs cyclotimia: pvalue 0.147
depressed vs bipolar: pvalue 0.917
depressed vs mania: pvalue 0.917
depressed vs cyclotimia: pvalue 0.054
bipolar vs mania: pvalue 0.864
bipolar vs cyclotimia: pvalue 0.030
mania vs cyclotimia: pvalue 0.025


In [94]:
# separate zones
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = hfds_og[new_labels_og == id]
    for zone in zone2ids.keys():
        zone_og = np.mean(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_og)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1

frontal: pvalue 0.058641002419835844
temporal: pvalue 0.0017451550817324899
central: pvalue 0.058641002419835844
parietal: pvalue 0.02179622164717914
occipital: pvalue 0.006932663525004479


In [99]:
diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
            pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
            print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
            cnt += 1
        print()

control vs depressed
	frontal: pvalue 0.139
	temporal: pvalue 0.037
	central: pvalue 0.305
	parietal: pvalue 0.331
	occipital: pvalue 0.037

control vs bipolar
	frontal: pvalue 0.170
	temporal: pvalue 0.023
	central: pvalue 0.272
	parietal: pvalue 0.139
	occipital: pvalue 0.021

control vs mania
	frontal: pvalue 0.190
	temporal: pvalue 0.018
	central: pvalue 0.743
	parietal: pvalue 0.177
	occipital: pvalue 0.080

control vs cyclotimia
	frontal: pvalue 0.257
	temporal: pvalue 0.555
	central: pvalue 0.138
	parietal: pvalue 0.125
	occipital: pvalue 0.901

depressed vs bipolar
	frontal: pvalue 0.955
	temporal: pvalue 0.955
	central: pvalue 0.975
	parietal: pvalue 0.955
	occipital: pvalue 0.895

depressed vs mania
	frontal: pvalue 0.955
	temporal: pvalue 0.595
	central: pvalue 0.903
	parietal: pvalue 0.973
	occipital: pvalue 0.955

depressed vs cyclotimia
	frontal: pvalue 0.093
	temporal: pvalue 0.075
	central: pvalue 0.037
	parietal: pvalue 0.021
	occipital: pvalue 0.139

bipolar vs mania


Closed eyes

In [75]:
# simple mean of all channels
groups = []

for diag, id in fldr2label.items():
    grp = hfds_zg[labels_zg == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=36.261678449565544, pvalue=6.4698198187972935e-06)


In [76]:
diags = list(fldr2label.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control (160) vs Биполярка депресс тип (35): pvalue 0.141
control (160) vs Биполярка маниакальный тип (36): pvalue 0.030
control (160) vs мания_психи(30): pvalue 0.020
control (160) vs циклотимия (13): pvalue 0.299
control (160) vs depressed_32-0(22): pvalue 0.880
control (160) vs depressed_32-1(44): pvalue 0.000
control (160) vs depressed_32-2(22): pvalue 0.723
Биполярка депресс тип (35) vs Биполярка маниакальный тип (36): pvalue 0.791
Биполярка депресс тип (35) vs мания_психи(30): pvalue 0.723
Биполярка депресс тип (35) vs циклотимия (13): pvalue 0.070
Биполярка депресс тип (35) vs depressed_32-0(22): pvalue 0.299
Биполярка депресс тип (35) vs depressed_32-1(44): pvalue 0.141
Биполярка депресс тип (35) vs depressed_32-2(22): pvalue 0.563
Биполярка маниакальный тип (36) vs мания_психи(30): pvalue 0.588
Биполярка маниакальный тип (36) vs циклотимия (13): pvalue 0.028
Биполярка маниакальный тип (36) vs depressed_32-0(22): pvalue 0.141
Биполярка маниакальный тип (36) vs depressed_32-1(44

In [85]:
# larger classes
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_zg = np.array([label_map[i] for i in labels_zg])

groups = []
for diag, id in new_classes.items():
    grp = hfds_zg[new_labels_zg == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=22.382405069445834, pvalue=0.0001681779443054103)


In [78]:
diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control vs depressed: pvalue 0.010
control vs bipolar: pvalue 0.010
control vs mania: pvalue 0.010
control vs cyclotimia: pvalue 0.305
depressed vs bipolar: pvalue 0.837
depressed vs mania: pvalue 0.585
depressed vs cyclotimia: pvalue 0.012
bipolar vs mania: pvalue 0.585
bipolar vs cyclotimia: pvalue 0.012
mania vs cyclotimia: pvalue 0.010


In [100]:
# separate zones
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = hfds_zg[new_labels_zg == id]
    for zone in zone2ids.keys():
        zone_zg = np.mean(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_zg)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1

frontal: pvalue 0.015615423341096023
temporal: pvalue 2.989826833002926e-05
central: pvalue 0.00024757152501673433
parietal: pvalue 7.510633387550245e-06
occipital: pvalue 0.00024757152501673433


In [101]:
diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
            pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
            print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
            cnt += 1
        print()

control vs depressed
	frontal: pvalue 0.049
	temporal: pvalue 0.002
	central: pvalue 0.012
	parietal: pvalue 0.005
	occipital: pvalue 0.004

control vs bipolar
	frontal: pvalue 0.044
	temporal: pvalue 0.005
	central: pvalue 0.014
	parietal: pvalue 0.012
	occipital: pvalue 0.023

control vs mania
	frontal: pvalue 0.044
	temporal: pvalue 0.002
	central: pvalue 0.012
	parietal: pvalue 0.004
	occipital: pvalue 0.116

control vs cyclotimia
	frontal: pvalue 0.551
	temporal: pvalue 0.662
	central: pvalue 0.190
	parietal: pvalue 0.020
	occipital: pvalue 0.269

depressed vs bipolar
	frontal: pvalue 0.920
	temporal: pvalue 0.794
	central: pvalue 0.949
	parietal: pvalue 0.802
	occipital: pvalue 0.574

depressed vs mania
	frontal: pvalue 0.574
	temporal: pvalue 0.459
	central: pvalue 0.534
	parietal: pvalue 0.679
	occipital: pvalue 0.631

depressed vs cyclotimia
	frontal: pvalue 0.116
	temporal: pvalue 0.027
	central: pvalue 0.012
	parietal: pvalue 0.002
	occipital: pvalue 0.002

bipolar vs mania


## Alpha rhythm

In [14]:
channels2use = ['Fp1', 'Fp2', 'F3', 'Fz', 'F4', 'F7', 'F8', 'T3', 'T4', 'C3', 'Cz', 'C4', 'T5', 'T6', 'P3', 'Pz', 'P4', 'O1', 'O2']
name_pairs = [('Fp1', 'Fp2'), ('F3', 'F4'), ('F7', 'F8'), ('C3', 'C4'), ('T3', 'T4'), ('P3', 'P4'), ('T5', 'T6'), ('O1', 'O2')]
idx_pairs = [(0, 1), (2, 4), (5, 6), (9, 11), (7, 8), (14, 16), (12, 13), (17, 18)]

def get_data_from_sample(sample):
    data = None
    sample = sample.filter(l_freq=1, h_freq=30, method='iir', verbose=False)
    channels = sample.ch_names
    to_drop = channels[19:]
    sample.drop_channels(to_drop)

    new_idx = []
    skip = False
    for ch in channels2use:
        found = False
        for k in range(19):
            if ch in channels[k]:
                new_idx.append(k)
                found = True
                break
        if not found:
            skip = True
            break
    if not skip:
        s_freq = int(sample.info['sfreq'])
        data = sample.get_data()[new_idx]
    return data

def compute_alpha_psd(data, s_freq):
    psd, freqs = mne.time_frequency.psd_array_welch(data, sfreq=s_freq, fmin=8, fmax=12, n_fft=1024, n_per_seg=s_freq, verbose=False)
    psd_avg = np.mean(psd, axis=1)
    return psd_avg

psds_og, psds_zg = [], []
labels_og, labels_zg = [], []
for i in tqdm(range(len(og_files))):
    path_og = og_files[i]
    path_zg = zg_files[i]
    try:
        sample_og = mne.io.read_raw_edf(path_og, verbose=False, preload=True)
    except Exception as e:
        print(f'skipped {path_og}')
        continue

    sample_zg = mne.io.read_raw_edf(path_zg, verbose=False, preload=True)
    s_freq_og = int(sample_og.info['sfreq'])
    s_freq_zg = int(sample_zg.info['sfreq'])
    data_og = get_data_from_sample(sample_og)
    data_zg = get_data_from_sample(sample_zg)
    if data_og is None or data_zg is None:
        print(f'skipped {path_og} and {path_zg}')
    else:
        max_len = max(data_og.shape[1], data_zg.shape[1])
        data_og = data_og[:, :max_len]
        data_zg = data_zg[:, :max_len]

        psd_og = compute_alpha_psd(data_og, s_freq_og)
        psd_zg = compute_alpha_psd(data_zg, s_freq_zg)
        psds_og.append(psd_og)
        psds_zg.append(psd_zg)
        labels_og.append(og_labels[i])
        labels_zg.append(zg_labels[i])

psds_og = np.array(psds_og)
psds_zg = np.array(psds_zg)
labels_og = np.array(labels_og)
labels_zg = np.array(labels_zg)

  0%|          | 0/363 [00:00<?, ?it/s]

skipped bipolar/control (160)/BORUTTO_JANNA_VLADIMIROVNA_48_EO_free.edf and bipolar/control (160)/BORUTTO_JANNA_VLADIMIROVNA_48_EC_free.edf
skipped bipolar/control (160)/MANUILOVA_ELENA_55_EO_free.edf and bipolar/control (160)/MANUILOVA_ELENA_55_EC_free.edf
skipped bipolar/control (160)/Martinenko_45_EO.edf and bipolar/control (160)/Martinenko_45_EC_free.edf
skipped bipolar/control (160)/Skopincev_20_EO_free.edf


### Open vs closed eyes

#### All electrodes

In [4]:
mw = stats.mannwhitneyu(np.array(psds_og), np.array(psds_zg))[1]
print(false_discovery_control(mw))

[5.85305239e-46 2.46376300e-49 1.36798869e-50 9.58947018e-52
 1.95868981e-52 3.48881630e-49 1.95868981e-52 2.90169162e-36
 3.10690864e-40 5.10150548e-42 6.15266068e-52 7.81851091e-45
 4.87508637e-51 4.49862133e-56 2.17910458e-53 1.75458648e-58
 1.03681953e-57 4.87641646e-56 2.89151245e-59]


In [6]:
mw = stats.mannwhitneyu(np.array(psds_og), np.array(psds_zg), alternative='less')[1]
print(false_discovery_control(mw))

print(f'MAPE by channels: {np.mean(np.abs(psds_og - psds_zg) / psds_og, axis=0) * 100}')

[2.92652619e-46 1.23188150e-49 6.83994345e-51 4.79473509e-52
 9.79344906e-53 1.74440815e-49 9.79344906e-53 1.45084581e-36
 1.55345432e-40 2.55075274e-42 3.07633034e-52 3.90925545e-45
 2.43754319e-51 2.24931066e-56 1.08955229e-53 8.77293242e-59
 5.18409765e-58 2.43820823e-56 1.44575623e-59]
MAPE by channels: [ 572.47970703  597.11297288  538.49805356  535.31244824  526.71734369
  554.63153268  569.30626578  391.77841777  396.60945239  488.8944954
  614.7340517   497.6184075   610.34738287  826.03986928  745.98891047
  803.22790041  871.28119449 1112.01968122  996.2107431 ]


#### Combined by zones

In [7]:
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

mws = []
for zone in zone2ids.keys():
    print(f'ZONE: {zone}')
    zone_og = np.mean(psds_og[:, zone2ids[zone]], axis=1)
    zone_zg = np.mean(psds_zg[:, zone2ids[zone]], axis=1)
    mw = stats.mannwhitneyu(zone_og, zone_zg, alternative='less')[1]
    mws.append(mw)
    print(f'MAPE: {np.mean(np.abs(zone_og - zone_zg) / zone_og, axis=0) * 100 :.3f}')
    print()

print(false_discovery_control(mws))

ZONE: frontal
MAPE: 505.659

ZONE: temporal
MAPE: 527.770

ZONE: central
MAPE: 485.594

ZONE: parietal
MAPE: 759.043

ZONE: occipital
MAPE: 971.994

[1.17940574e-53 1.22252911e-57 3.24664360e-49 2.23242881e-60
 1.20274702e-61]


#### Left vs right vs center

In [9]:
side2ids = {'right': [1, 4, 6, 8, 11, 13, 16, 18], 'left': [0, 2, 5, 7, 9, 12, 14, 17], 'center': [3, 10, 15]}
mws = []
for side in side2ids.keys():
    side_og = np.mean(psds_og[:, side2ids[side]], axis=1)
    side_zg = np.mean(psds_zg[:, side2ids[side]], axis=1)
    mw = stats.mannwhitneyu(side_og, side_zg, alternative='less')[1]
    mws.append(mw)
    print(f'MAPE: {np.mean(np.abs(side_og - side_zg) / side_og, axis=0) * 100 :.3f}')

print(false_discovery_control(mws))

MAPE: 594.044
MAPE: 543.505
MAPE: 597.092
[1.26808551e-61 1.66383578e-57 2.20224387e-59]


#### Avg

In [10]:
avg_og = np.mean(psds_og, axis=1)
avg_zg = np.mean(psds_zg, axis=1)
mw = stats.mannwhitneyu(avg_og, avg_zg, alternative='less')[1]
print(mw)
print(f'MAPE: {np.mean(np.abs(avg_og - avg_zg) / avg_og, axis=0) * 100 :.3f}')

3.255278897702953e-61
MAPE: 559.116


#### Max

In [11]:
avg_og = np.max(psds_og, axis=1)
avg_zg = np.max(psds_zg, axis=1)
mw = stats.mannwhitneyu(avg_og, avg_zg, alternative='less')[1]
print(mw)
print(f'MAPE: {np.mean(np.abs(avg_og - avg_zg) / avg_og, axis=0) * 100 :.3f}')

2.2382811800242625e-62
MAPE: 658.767


### By diagnoses

Open eyes

In [15]:
# simple mean of all channels
groups = []

for diag, id in fldr2label.items():
    grp = psds_og[labels_og == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=27.918905342867, pvalue=0.0002274579002556021)


In [16]:
diags = list(fldr2label.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control (160) vs Биполярка депресс тип (35): pvalue 0.220
control (160) vs Биполярка маниакальный тип (36): pvalue 0.045
control (160) vs мания_психи(30): pvalue 0.045
control (160) vs циклотимия (13): pvalue 0.088
control (160) vs depressed_32-0(22): pvalue 0.003
control (160) vs depressed_32-1(44): pvalue 0.231
control (160) vs depressed_32-2(22): pvalue 0.410
Биполярка депресс тип (35) vs Биполярка маниакальный тип (36): pvalue 0.729
Биполярка депресс тип (35) vs мания_психи(30): pvalue 0.626
Биполярка депресс тип (35) vs циклотимия (13): pvalue 0.257
Биполярка депресс тип (35) vs depressed_32-0(22): pvalue 0.220
Биполярка депресс тип (35) vs depressed_32-1(44): pvalue 0.788
Биполярка депресс тип (35) vs depressed_32-2(22): pvalue 0.785
Биполярка маниакальный тип (36) vs мания_психи(30): pvalue 0.727
Биполярка маниакальный тип (36) vs циклотимия (13): pvalue 0.380
Биполярка маниакальный тип (36) vs depressed_32-0(22): pvalue 0.214
Биполярка маниакальный тип (36) vs depressed_32-1(44

In [17]:
# larger classes
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_og = np.array([label_map[i] for i in labels_og])

groups = []
for diag, id in new_classes.items():
    grp = psds_og[new_labels_og == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=20.893136596687064, pvalue=0.00033249746392158925)


In [18]:
diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control vs depressed: pvalue 0.013
control vs bipolar: pvalue 0.013
control vs mania: pvalue 0.021
control vs cyclotimia: pvalue 0.039
depressed vs bipolar: pvalue 0.983
depressed vs mania: pvalue 0.541
depressed vs cyclotimia: pvalue 0.233
bipolar vs mania: pvalue 0.541
bipolar vs cyclotimia: pvalue 0.233
mania vs cyclotimia: pvalue 0.478


In [19]:
# larger classes, max power
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_og = np.array([label_map[i] for i in labels_og])

groups = []
for diag, id in new_classes.items():
    grp = psds_og[new_labels_og == id].max(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=25.672466560270983, pvalue=3.683940385067916e-05)


In [20]:
diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control vs depressed: pvalue 0.004
control vs bipolar: pvalue 0.004
control vs mania: pvalue 0.009
control vs cyclotimia: pvalue 0.011
depressed vs bipolar: pvalue 0.895
depressed vs mania: pvalue 0.511
depressed vs cyclotimia: pvalue 0.173
bipolar vs mania: pvalue 0.399
bipolar vs cyclotimia: pvalue 0.173
mania vs cyclotimia: pvalue 0.418


In [21]:
# separate zones
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = psds_og[new_labels_og == id]
    for zone in zone2ids.keys():
        zone_og = np.mean(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_og)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1

frontal: pvalue 0.01946683968612112
temporal: pvalue 0.002974680359898132
central: pvalue 0.0004606181402992281
parietal: pvalue 2.9107824826164582e-06
occipital: pvalue 0.1551791132723925


In [22]:
diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
            if zone != 'occipital':
                pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
            if zone != 'occipital':
                print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
                cnt += 1
        print()

control vs depressed
	frontal: pvalue 0.034
	temporal: pvalue 0.025
	central: pvalue 0.006
	parietal: pvalue 0.002

control vs bipolar
	frontal: pvalue 0.063
	temporal: pvalue 0.022
	central: pvalue 0.025
	parietal: pvalue 0.002

control vs mania
	frontal: pvalue 0.072
	temporal: pvalue 0.028
	central: pvalue 0.025
	parietal: pvalue 0.025

control vs cyclotimia
	frontal: pvalue 0.151
	temporal: pvalue 0.151
	central: pvalue 0.026
	parietal: pvalue 0.002

depressed vs bipolar
	frontal: pvalue 0.921
	temporal: pvalue 0.763
	central: pvalue 0.921
	parietal: pvalue 0.959

depressed vs mania
	frontal: pvalue 0.604
	temporal: pvalue 0.586
	central: pvalue 0.594
	parietal: pvalue 0.808

depressed vs cyclotimia
	frontal: pvalue 0.586
	temporal: pvalue 0.586
	central: pvalue 0.323
	parietal: pvalue 0.028

bipolar vs mania
	frontal: pvalue 0.586
	temporal: pvalue 0.725
	central: pvalue 0.586
	parietal: pvalue 0.921

bipolar vs cyclotimia
	frontal: pvalue 0.586
	temporal: pvalue 0.586
	central: p

In [23]:
# separate zones, max
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = psds_og[new_labels_og == id]
    for zone in zone2ids.keys():
        zone_og = np.max(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_og)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1

diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
            if zone != 'occipital':
                pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
            if zone != 'occipital':
                print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
                cnt += 1
        print()

frontal: pvalue 0.009865157635055978
temporal: pvalue 0.008534508693794868
central: pvalue 5.468764934207931e-05
parietal: pvalue 1.048158007811026e-06
occipital: pvalue 0.10902473256924641
control vs depressed
	frontal: pvalue 0.031
	temporal: pvalue 0.027
	central: pvalue 0.002
	parietal: pvalue 0.001

control vs bipolar
	frontal: pvalue 0.062
	temporal: pvalue 0.027
	central: pvalue 0.011
	parietal: pvalue 0.001

control vs mania
	frontal: pvalue 0.044
	temporal: pvalue 0.044
	central: pvalue 0.012
	parietal: pvalue 0.012

control vs cyclotimia
	frontal: pvalue 0.148
	temporal: pvalue 0.233
	central: pvalue 0.027
	parietal: pvalue 0.002

depressed vs bipolar
	frontal: pvalue 0.823
	temporal: pvalue 0.744
	central: pvalue 0.744
	parietal: pvalue 0.983

depressed vs mania
	frontal: pvalue 0.624
	temporal: pvalue 0.643
	central: pvalue 0.643
	parietal: pvalue 0.744

depressed vs cyclotimia
	frontal: pvalue 0.591
	temporal: pvalue 0.634
	central: pvalue 0.373
	parietal: pvalue 0.052

bi

Closed eyes

In [24]:
# simple mean of all channels
groups = []

for diag, id in fldr2label.items():
    grp = psds_zg[labels_zg == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=13.331762916889852, pvalue=0.06442594617835447)


In [25]:
diags = list(fldr2label.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control (160) vs Биполярка депресс тип (35): pvalue 0.954
control (160) vs Биполярка маниакальный тип (36): pvalue 0.397
control (160) vs мания_психи(30): pvalue 0.578
control (160) vs циклотимия (13): pvalue 0.578
control (160) vs depressed_32-0(22): pvalue 0.414
control (160) vs depressed_32-1(44): pvalue 0.399
control (160) vs depressed_32-2(22): pvalue 0.728
Биполярка депресс тип (35) vs Биполярка маниакальный тип (36): pvalue 0.363
Биполярка депресс тип (35) vs мания_психи(30): pvalue 0.578
Биполярка депресс тип (35) vs циклотимия (13): pvalue 0.578
Биполярка депресс тип (35) vs depressed_32-0(22): pvalue 0.399
Биполярка депресс тип (35) vs depressed_32-1(44): pvalue 0.440
Биполярка депресс тип (35) vs depressed_32-2(22): pvalue 0.734
Биполярка маниакальный тип (36) vs мания_психи(30): pvalue 0.728
Биполярка маниакальный тип (36) vs циклотимия (13): pvalue 0.728
Биполярка маниакальный тип (36) vs depressed_32-0(22): pvalue 0.728
Биполярка маниакальный тип (36) vs depressed_32-1(44

Note Биполярка маниакальный тип (36) vs depressed_32-1(44): pvalue 0.018: the data from the open eyes does not capture this

In [26]:
# larger classes
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_zg = np.array([label_map[i] for i in labels_zg])

groups = []
for diag, id in new_classes.items():
    grp = psds_zg[new_labels_zg == id].mean(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

KruskalResult(statistic=3.578624251164102, pvalue=0.46602448183129397)


In [27]:
diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

control vs depressed: pvalue 0.883
control vs bipolar: pvalue 0.567
control vs mania: pvalue 0.567
control vs cyclotimia: pvalue 0.567
depressed vs bipolar: pvalue 0.567
depressed vs mania: pvalue 0.567
depressed vs cyclotimia: pvalue 0.567
bipolar vs mania: pvalue 0.877
bipolar vs cyclotimia: pvalue 0.567
mania vs cyclotimia: pvalue 0.735


In [28]:
# larger classes, max power
label_map = {0: 0, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4}
new_classes = {'control': 0, 'depressed': 1, 'bipolar': 2, 'mania': 3, 'cyclotimia': 4}
new_labels_zg = np.array([label_map[i] for i in labels_zg])

groups = []
for diag, id in new_classes.items():
    grp = psds_zg[new_labels_zg == id].max(axis=1)
    groups.append(grp)

print(stats.kruskal(*groups))

diags = list(new_classes.keys())
pvals = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        pvals.append(stats.mannwhitneyu(groups[i], groups[j])[1])

pvals_corrected = false_discovery_control(pvals)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}: pvalue {pvals_corrected[cnt]:.3f}')
        cnt += 1

KruskalResult(statistic=4.557255433232086, pvalue=0.3358123487660972)
control vs depressed: pvalue 0.942
control vs bipolar: pvalue 0.545
control vs mania: pvalue 0.545
control vs cyclotimia: pvalue 0.575
depressed vs bipolar: pvalue 0.545
depressed vs mania: pvalue 0.545
depressed vs cyclotimia: pvalue 0.575
bipolar vs mania: pvalue 0.635
bipolar vs cyclotimia: pvalue 0.587
mania vs cyclotimia: pvalue 0.653


In [29]:
# separate zones
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = psds_zg[new_labels_zg == id]
    for zone in zone2ids.keys():
        zone_zg = np.mean(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_zg)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1

frontal: pvalue 0.46296848949537883
temporal: pvalue 0.46296848949537883
central: pvalue 0.46296848949537883
parietal: pvalue 0.0827237268977804
occipital: pvalue 0.0827237268977804


In [30]:
diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
                pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
                print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
                cnt += 1
        print()

control vs depressed
	frontal: pvalue 0.850
	temporal: pvalue 0.543
	central: pvalue 0.850
	parietal: pvalue 0.850
	occipital: pvalue 0.140

control vs bipolar
	frontal: pvalue 0.508
	temporal: pvalue 0.850
	central: pvalue 0.521
	parietal: pvalue 0.442
	occipital: pvalue 0.521

control vs mania
	frontal: pvalue 0.543
	temporal: pvalue 0.747
	central: pvalue 0.747
	parietal: pvalue 0.684
	occipital: pvalue 0.968

control vs cyclotimia
	frontal: pvalue 0.930
	temporal: pvalue 0.747
	central: pvalue 0.442
	parietal: pvalue 0.080
	occipital: pvalue 0.684

depressed vs bipolar
	frontal: pvalue 0.508
	temporal: pvalue 0.442
	central: pvalue 0.563
	parietal: pvalue 0.521
	occipital: pvalue 0.531

depressed vs mania
	frontal: pvalue 0.538
	temporal: pvalue 0.442
	central: pvalue 0.850
	parietal: pvalue 0.684
	occipital: pvalue 0.412

depressed vs cyclotimia
	frontal: pvalue 0.850
	temporal: pvalue 0.563
	central: pvalue 0.521
	parietal: pvalue 0.080
	occipital: pvalue 0.273

bipolar vs mania


In [31]:
# separate zones, max power
zone2ids = {'frontal': [0, 1, 2, 3, 4, 5, 6], 'temporal': [7, 8, 12, 13], 'central': [9, 10, 11], 'parietal': [14, 15, 16], 'occipital': [17, 18]}

groups = {x: [] for x in zone2ids.keys()}
for diag, id in new_classes.items():
    grp = psds_zg[new_labels_zg == id]
    for zone in zone2ids.keys():
        zone_zg = np.max(grp[:, zone2ids[zone]], axis=1)
        groups[zone].append(zone_zg)

pvals = []
for zone in zone2ids.keys():
    pvals.append(stats.kruskal(*groups[zone])[1])
pvals_corrected = false_discovery_control(pvals)

cnt = 0
for zone in zone2ids.keys():
    print(f'{zone}: pvalue {pvals_corrected[cnt]}')
    cnt += 1
print()

diags = list(new_classes.keys())

pvals_unwrapped = []
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        for zone in zone2ids.keys():
                pvals_unwrapped.append(stats.mannwhitneyu(groups[zone][i], groups[zone][j])[1])

pvals_corrected = false_discovery_control(pvals_unwrapped)
cnt = 0
for i in range(len(diags)):
    for j in range(i + 1, len(diags)):
        print(f'{diags[i]} vs {diags[j]}')
        for zone in zone2ids.keys():
                print(f'\t{zone}: pvalue {pvals_corrected[cnt]:.3f}')
                cnt += 1
        print()

frontal: pvalue 0.49438520561255517
temporal: pvalue 0.39681027424315196
central: pvalue 0.39681027424315196
parietal: pvalue 0.1421713929912354
occipital: pvalue 0.1421713929912354

control vs depressed
	frontal: pvalue 0.823
	temporal: pvalue 0.497
	central: pvalue 0.624
	parietal: pvalue 0.660
	occipital: pvalue 0.400

control vs bipolar
	frontal: pvalue 0.497
	temporal: pvalue 0.952
	central: pvalue 0.440
	parietal: pvalue 0.400
	occipital: pvalue 0.600

control vs mania
	frontal: pvalue 0.600
	temporal: pvalue 0.668
	central: pvalue 0.600
	parietal: pvalue 0.600
	occipital: pvalue 0.940

control vs cyclotimia
	frontal: pvalue 0.940
	temporal: pvalue 0.654
	central: pvalue 0.426
	parietal: pvalue 0.400
	occipital: pvalue 0.938

depressed vs bipolar
	frontal: pvalue 0.497
	temporal: pvalue 0.497
	central: pvalue 0.660
	parietal: pvalue 0.497
	occipital: pvalue 0.497

depressed vs mania
	frontal: pvalue 0.600
	temporal: pvalue 0.426
	central: pvalue 0.932
	parietal: pvalue 0.660
	occ